In [1]:
"""
    IMPORTING LIBS
"""
import dgl

import numpy as np
import os
import time
import random

import torch
from utils.notebook_utils import Args

Using backend: pytorch


In [2]:
#   Set manual seed

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.random.manual_seed(seed)
    dgl.random.seed(seed)
    if torch.cuda.is_available():
       torch.cuda.manual_seed(seed)
       torch.cuda.manual_seed_all(seed)
       torch.backends.cudnn.deterministic = True
       torch.backends.cudnn.benchmark = False
        
"""
    VIEWING MODEL CONFIG AND PARAMS
"""
def view_model_param(MODEL_NAME, net_params):
    model = gnn_model(MODEL_NAME, net_params)
    model = model.float()
    total_param = 0
    print("MODEL DETAILS:\n")
    print(model)
    for param in model.parameters():
        print(param.data.size())
        total_param += np.prod(list(param.data.size()))
    print('MODEL/Total parameters:', MODEL_NAME, total_param)
    return total_param

In [3]:
"""
    IMPORTING CUSTOM MODULES/METHODS
"""

from data.data import load_data # import dataset
from utils.utils import get_configs, get_arguments, add_dir_name # import arguments and configurations
from nets.molecules_graph_regression.load_net import gnn_model # import all GNNS

RDKit WARNING: [09:27:13] Enabling RDKit 2019.09.3 jupyter extensions


In [4]:
def main(args):    
    """
        USER CONTROLS
    """
    #args = get_arguments()

    args, config, params, net_params = get_configs(args)

    #   define which pipeline to be used
    if params['swa'] == True:
        from pipeline_swa import train_val_pipeline_classification
    elif params['swag'] == True:
        from pipeline_swag import train_val_pipeline_classification
    elif (params['sgld'] == True) or (params['psgld'] == True):
        from pipeline_sgld import train_val_pipeline_classification
    else:
        if params['bbp'] == True:
            from nets.molecules_graph_regression.load_bbp_net import gnn_model # import all GNNS
        from pipeline import train_val_pipeline_classification


    DATASET_NAME = config['dataset']
    MODEL_NAME = config['model']

    # setting seeds
    set_seed(params['seed'])
    print("Seed Number of Models: "+str(params['seed']))
    print("Data Seed Number: "+str(params['data_seed']))


    dataset = load_data(DATASET_NAME, args.num_train, args.num_val, args.num_test, args.data_seed, params)

    # network parameters

    #   add task information for net_params loss
    net_params['task'] = 'classification'
    net_params['num_classes'] = dataset.num_classes
    net_params['num_atom_type'] = dataset.num_atom_type
    net_params['num_bond_type'] = dataset.num_bond_type
    out_dir = config['out_dir']

    root_ckpt_dir = out_dir + 'checkpoints/' + MODEL_NAME + "_" + DATASET_NAME + "_GPU" +\
        str(config['gpu']['id']) + "_" + time.strftime('%Hh%Mm%Ss_on_%b_%d_%Y')
    write_file_name = out_dir + 'results/result_' + MODEL_NAME + "_" + DATASET_NAME + "_GPU" +\
        str(config['gpu']['id']) + "_" + time.strftime('%Hh%Mm%Ss_on_%b_%d_%Y')
    root_output_dir = out_dir + 'outputs/outputs_' + MODEL_NAME + "_" + DATASET_NAME + "_GPU" +\
        str(config['gpu']['id']) + "_" + time.strftime('%Hh%Mm%Ss_on_%b_%d_%Y')

    dirs = root_ckpt_dir, write_file_name, root_output_dir
    dirs = add_dir_name(dirs, MODEL_NAME, config, params, net_params)

    net_params['total_param'] = view_model_param(MODEL_NAME, net_params)
    train_val_pipeline_classification(MODEL_NAME, DATASET_NAME, dataset, config, params, net_params, dirs)

##### To run base GIN model on BACE dataset,

In [5]:
if __name__ == "__main__":
    
    config = 'configs/gin_bace.json'
    
    args = Args(config)
    
    args.gpu_id = 0
    main(args)


cuda available with GPU: TITAN Xp
True
Seed Number of Models: 41
Data Seed Number: 222
Seed Number of Data: 222
train/valid scaffold split:  False
train/test scaffold split:  False
valid/test scaffold split:  True
preparing 1200 graphs for the TRAIN set...
preparing 150 graphs for the VALIDATION set...
preparing 150 graphs for the TEST set...
Time taken: 9.4213s
MODEL DETAILS:

GINNet(
  (ginlayers): ModuleList(
    (0): GINLayer(
      (apply_func): ApplyNodeFunc(
        (mlp): MLP(
          (linear_1): Linear(in_features=128, out_features=128, bias=False)
          (linear_2): Linear(in_features=128, out_features=128, bias=False)
        )
      )
    )
    (1): GINLayer(
      (apply_func): ApplyNodeFunc(
        (mlp): MLP(
          (linear_1): Linear(in_features=128, out_features=128, bias=False)
          (linear_2): Linear(in_features=128, out_features=128, bias=False)
        )
      )
    )
    (2): GINLayer(
      (apply_func): ApplyNodeFunc(
        (mlp): MLP(
          

Epoch 0:   0%|          | 0/200 [00:00<?, ?it/s]/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
Epoch 1:   0%|          | 1/200 [00:00<01:27,  2.29it/s, lr=0.001, test_AUC=0.652, test_EC

/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
Epoch 7:   4%|▎         | 7/200 [00:01<00:47,  4.08it/s, lr=0.001, test_AUC=0.598, test_ECE=0.0695, time=0.214, train_AUC=0.617, train_ECE

Epoch 13:   6%|▋         | 13/200 [00:03<00:45,  4.13it/s, lr=0.001, test_AUC=0.668, test_ECE=0.179, time=0.267, train_AUC=0.722, train_ECE=0.0272, train_loss=0.606, val_AUC=0.67, val_ECE=0.26, val_loss=0.67]    /home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43

Epoch 97:  48%|████▊     | 97/200 [00:21<00:22,  4.66it/s, lr=0.0001, test_AUC=0.898, test_ECE=0.103, time=0.206, train_AUC=0.974, train_ECE=0.0404, train_loss=0.212, val_AUC=0.832, val_ECE=0.154, val_loss=0.832]/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
Epoch 99:  50%|████▉     | 99/200 [00:21<00:21,  4.66it/s, lr=0.0001, test_AUC=0.899, test_ECE=0.109, time=0.211, train_AUC=0.974, train_ECE=0.0329, train_loss=0.202, val_AUC=0.825, val_ECE=0.161, val_loss=0.825]/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/env

Epoch 199: 100%|██████████| 200/200 [00:43<00:00,  4.57it/s, lr=1e-5, test_AUC=0.903, test_ECE=0.129, time=0.215, train_AUC=0.985, train_ECE=0.0335, train_loss=0.159, val_AUC=0.819, val_ECE=0.157, val_loss=0.819]  

Test AUC: 0.9025
Test ECE: 0.1293
Train AUC: 0.9852
Train ECE: 0.0349
TOTAL TIME TAKEN: 47.1590s
AVG TIME PER EPOCH: 0.2176s


##### To run bace GIN model on BBBP dataset,

In [6]:
if __name__ == "__main__":
    
    config = 'configs/gin_bbbp.json'
    
    args = Args(config)
    
    args.gpu_id = 0
    main(args)

cuda available with GPU: TITAN Xp
True
Seed Number of Models: 41
Data Seed Number: 222
Seed Number of Data: 222


RDKit WARNING: [09:28:19] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:19] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:19] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:19] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:19] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:19] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:20] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:20] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:20] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:20] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:20] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:20] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:20] WARNING: not r

train/valid scaffold split:  False
train/test scaffold split:  False
valid/test scaffold split:  False
preparing 1600 graphs for the TRAIN set...


RDKit WARNING: [09:28:21] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:21] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:22] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:22] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:22] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:22] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:23] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:23] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:23] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:23] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:23] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:23] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:28:23] WARNING: not r

preparing 200 graphs for the VALIDATION set...


...

Epoch 176:  88%|████████▊ | 176/200 [00:46<00:06,  3.81it/s, lr=1e-5, test_AUC=0.871, test_ECE=0.169, time=0.261, train_AUC=1, train_ECE=0.00614, train_loss=0.00829, val_AUC=0.91, val_ECE=0.133, val_loss=0.91]/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: R

Epoch 185:  92%|█████████▎| 185/200 [00:49<00:04,  3.54it/s, lr=1e-5, test_AUC=0.87, test_ECE=0.169, time=0.322, train_AUC=1, train_ECE=0.00547, train_loss=0.00816, val_AUC=0.91, val_ECE=0.133, val_loss=0.91] /home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: R

Epoch 193:  96%|█████████▋| 193/200 [00:51<00:01,  3.83it/s, lr=1e-5, test_AUC=0.87, test_ECE=0.169, time=0.254, train_AUC=1, train_ECE=0.00556, train_loss=0.00804, val_AUC=0.91, val_ECE=0.13, val_loss=0.91] /home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: Ru

Epoch 199: 100%|██████████| 200/200 [00:53<00:00,  3.76it/s, lr=1e-5, test_AUC=0.871, test_ECE=0.17, time=0.265, train_AUC=1, train_ECE=0.00571, train_loss=0.00762, val_AUC=0.91, val_ECE=0.13, val_loss=0.91]   
/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()


Test AUC: 0.8714
Test ECE: 0.1698
Train AUC: 1.0000
Train ECE: 0.0055
TOTAL TIME TAKEN: 53.3558s
AVG TIME PER EPOCH: 0.2645s


##### To run base GCN model on BACE dataset,

In [7]:
if __name__ == "__main__":
    
    config = 'configs/gcn_bace.json'
    
    args = Args(config)
    
    args.gpu_id = 0
    main(args)

cuda available with GPU: TITAN Xp
True
Seed Number of Models: 41
Data Seed Number: 222
Seed Number of Data: 222
train/valid scaffold split:  False
train/test scaffold split:  False
valid/test scaffold split:  True
preparing 1200 graphs for the TRAIN set...
preparing 150 graphs for the VALIDATION set...
preparing 150 graphs for the TEST set...
Time taken: 8.5771s
MODEL DETAILS:

GCNNet(
  (in_feat_dropout): Dropout(p=0.0, inplace=False)
  (embedding_lin): Linear(in_features=58, out_features=128, bias=False)
  (layers): ModuleList(
    (0): GCNLayer(in_channels=128, out_channels=128, residual=True)
    (1): GCNLayer(in_channels=128, out_channels=128, residual=True)
    (2): GCNLayer(in_channels=128, out_channels=128, residual=True)
    (3): GCNLayer(in_channels=128, out_channels=128, residual=True)
  )
  (linear_ro): Linear(in_features=128, out_features=256, bias=False)
  (linear_predict): Linear(in_features=256, out_features=1, bias=True)
)
torch.Size([128, 58])
torch.Size([128, 128])
t

Epoch 0:   0%|          | 0/200 [00:00<?, ?it/s]

Training Graphs:  1200
Validation Graphs:  150
Test Graphs:  150


/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anacon

Epoch 6:   3%|▎         | 6/200 [00:02<01:14,  2.61it/s, lr=0.001, test_AUC=0.653, test_ECE=0.0511, time=0.381, train_AUC=0.581, train_ECE=0.0283, train_loss=0.673, val_AUC=0.708, val_ECE=0.103, val_loss=0.708]/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: 

Epoch 12:   6%|▌         | 12/200 [00:04<01:11,  2.62it/s, lr=0.001, test_AUC=0.689, test_ECE=0.0768, time=0.377, train_AUC=0.694, train_ECE=0.0604, train_loss=0.636, val_AUC=0.773, val_ECE=0.145, val_loss=0.773]/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43

Epoch 19:  10%|▉         | 19/200 [00:07<01:08,  2.64it/s, lr=0.001, test_AUC=0.747, test_ECE=0.083, time=0.377, train_AUC=0.745, train_ECE=0.0475, train_loss=0.591, val_AUC=0.807, val_ECE=0.108, val_loss=0.807]/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43:

Epoch 27:  14%|█▎        | 27/200 [00:10<01:06,  2.58it/s, lr=0.001, test_AUC=0.8, test_ECE=0.148, time=0.379, train_AUC=0.802, train_ECE=0.0357, train_loss=0.541, val_AUC=0.818, val_ECE=0.141, val_loss=0.818]/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: R

Epoch 40:  20%|██        | 40/200 [00:15<01:01,  2.60it/s, lr=0.001, test_AUC=0.859, test_ECE=0.167, time=0.378, train_AUC=0.792, train_ECE=0.0592, train_loss=0.566, val_AUC=0.843, val_ECE=0.132, val_loss=0.843]/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
Epoch 42:  21%|██        | 42/200 [00:16<01:01,  2.59it/s, lr=0.001, test_AUC=0.871, test_ECE=0.154, time=0.391, train_AUC=0.837, train_ECE=0.0625, train_loss=0.503, val_AUC=0.841, val_ECE=0.121, val_loss=0.841]/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/

Test AUC: 0.8819
Test ECE: 0.0763
Train AUC: 0.9461
Train ECE: 0.0442
TOTAL TIME TAKEN: 91.7025s
AVG TIME PER EPOCH: 0.4522s


##### To run GIN SWAG model on BACE dataset,

In [8]:
if __name__ == "__main__":
    
    config = 'configs/gin_bace.json'
    
    args = Args(config)
    
    args.gpu_id = 0
    args.swag = True
    args.epochs = 250
    main(args)

cuda available with GPU: TITAN Xp
True
Seed Number of Models: 41
Data Seed Number: 222
Seed Number of Data: 222
train/valid scaffold split:  False
train/test scaffold split:  False
valid/test scaffold split:  True
preparing 1200 graphs for the TRAIN set...
preparing 150 graphs for the VALIDATION set...
preparing 150 graphs for the TEST set...


Epoch 0:   0%|          | 0/250 [00:00<?, ?it/s]

Time taken: 8.5988s
MODEL DETAILS:

GINNet(
  (ginlayers): ModuleList(
    (0): GINLayer(
      (apply_func): ApplyNodeFunc(
        (mlp): MLP(
          (linear_1): Linear(in_features=128, out_features=128, bias=False)
          (linear_2): Linear(in_features=128, out_features=128, bias=False)
        )
      )
    )
    (1): GINLayer(
      (apply_func): ApplyNodeFunc(
        (mlp): MLP(
          (linear_1): Linear(in_features=128, out_features=128, bias=False)
          (linear_2): Linear(in_features=128, out_features=128, bias=False)
        )
      )
    )
    (2): GINLayer(
      (apply_func): ApplyNodeFunc(
        (mlp): MLP(
          (linear_1): Linear(in_features=128, out_features=128, bias=False)
          (linear_2): Linear(in_features=128, out_features=128, bias=False)
        )
      )
    )
    (3): GINLayer(
      (apply_func): ApplyNodeFunc(
        (mlp): MLP(
          (linear_1): Linear(in_features=128, out_features=128, bias=False)
          (linear_2): Linear(

/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
Epoch 1:   0%|          | 1/250 [00:00<00:52,  4.77it/s, lr=0.001, test_AUC=0.652, test_ECE=0.249, time=0.208, train_AUC=0.546, train_ECE=

/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
Epoch 7:   3%|▎         | 7/250 [00:01<00:51,  4.71it/s, lr=0.001, test_AUC=0.597, test_ECE=0.0699, time=0.21, train_AUC=0.617, train_ECE=0.0356, train_loss=0.665, val_AUC=0.708, val_ECE=0.0914, val_loss=0.708] /home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43:

/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
Epoch 14:   6%|▌         | 14/250 [00:02<00:50,  4.70it/s, lr=0.001, test_AUC=0.795, test_ECE=0.14, time=0.21, train_AUC=0.715, train_ECE=0.0428, train_loss=0.617, val_AUC=0.785, val_ECE=0.127, val_loss=0.785]  /home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43:

Epoch 249: 100%|██████████| 250/250 [00:54<00:00,  4.61it/s, lr=1e-6, test_AUC=0.894, test_ECE=0.16, time=0.219, train_AUC=0.986, train_ECE=0.0354, train_loss=0.163, val_AUC=0.821, val_ECE=0.187, val_loss=0.821] 

Test AUC: 0.8943
Test ECE: 0.1603
Train AUC: 0.9858
Train ECE: 0.0360
TOTAL TIME TAKEN: 54.3650s
AVG TIME PER EPOCH: 0.2154s
